In [3]:
from datasets import load_dataset
import datasets
from transformers import AutoTokenizer, DataCollatorWithPadding
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
dataset = load_dataset('csv', data_files={'train': ['para_train.csv'], 'test': 'para_test.csv', 'validation':'para_validate.csv'})

Using custom data configuration default-cde8372ac5f3dd32
Reusing dataset csv (/home/dnarendr/.cache/huggingface/datasets/csv/default-cde8372ac5f3dd32/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
def tokenize_function(example):
    try:
      return tokenizer(example["text"], truncation=True)
    except  Exception as e:
      print(e)

In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print("tokenized_datasets::", tokenized_datasets)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

tokenized_datasets:: DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2027
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 272
    })
})


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="paraModel", evaluation_strategy="steps", eval_steps=100, num_train_epochs=3)

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [12]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def compute_metrics(eval_pred):    
    pred, labels = eval_pred
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=200)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/dnarendr/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19"

In [14]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/dnarendr/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2027
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 762


Step,Training Loss,Validation Loss


IndexError: Target 130 is out of bounds.

In [13]:
model.save_pretrained("my_model")

Configuration saved in my_model/config.json
Model weights saved in my_model/pytorch_model.bin


In [14]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('my_model', local_files_only=True)
classifier = pipeline(task = 'text-classification', model = model, tokenizer=tokenizer)


loading configuration file my_model/config.json
Model config BertConfig {
  "_name_or_path": "my_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights f

In [15]:
classifier(["Your Zions First National Bank Card has been issued by Zions Bancorporation, NA dba Zions First National Bank.", "Your Account is with Zions Bancorporation, NA (Bank) and will be administered by the Banks Bankcard Services department.", "Your Credit Card, monthly statement, and other associated materials will bear the Zions First National Bank, National Bank of Arizona, Nevada State Bank, or Vectra Bank Colorado name.", "The Card is the property of the Bank."])

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'LABEL_1', 'score': 0.9908129572868347},
 {'label': 'LABEL_1', 'score': 0.9354974031448364},
 {'label': 'LABEL_0', 'score': 0.7992977499961853},
 {'label': 'LABEL_1', 'score': 0.8896874189376831}]

In [16]:
model_predictions = trainer.predict(tokenized_datasets['test'])
import numpy as np

preds = np.argmax(model_predictions.predictions, axis=-1)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1679
  Batch size = 8


In [25]:
final_score = metric.compute(model_predictions=preds, references='model_predictions.label')

TypeError: 'NoneType' object is not iterable